# Desiertos de cuidado en Lima metropolitana y Callao
## Estadísticas descriptivas

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import geopandas as gpd
import numpy as np
import pandas as pd
import shapely
import plotly.express as px
import osmnx as ox

## - Personas con discapacidad

Importar bbdd y mergear 

In [2]:
gdf = gpd.read_file('results - desiertos/desiertos_ajustados/desiertos_disc_recalc.geojson')


In [3]:
gdf.columns

Index(['Mz', 'UBIGEO', 'AREA', 'CODCCPP', 'ZONA', 'MANZANA_ID', 'MANZANA_A',
       'pobdisc_mz', 'distancia_a_centro_disc', 'duracion_a_centro_disc',
       'alta_demanda', 'desierto_disc_recalc', 'umbral_t_dinamico',
       'geometry'],
      dtype='object')

In [4]:

manzanas = gpd.read_file('data/lima_manzanas.gpkg')


In [5]:
gdf = gdf.drop(columns='geometry')

In [6]:
pobreza = pd.read_csv("data/BBDD pobreza/peru_estratos_ingresos.csv")
print(pobreza.shape)
pobreza.head()



(98911, 10)


,FID,idmanzana,Ubigeo,Departamento,Provincia,Distrito,Hogares,Población,Estrato,geom
0,peru_estratos_ingresos.14645,150106092000020,150106,LIMA,LIMA,CARABAYLLO,4,10,2,MULTIPOLYGON (((-77.02289224399993 -11.8951647...
1,peru_estratos_ingresos.14646,150106092000010,150106,LIMA,LIMA,CARABAYLLO,31,129,2,MULTIPOLYGON (((-77.02352222499997 -11.8947903...
2,peru_estratos_ingresos.14647,150106035000100,150106,LIMA,LIMA,CARABAYLLO,14,50,1,MULTIPOLYGON (((-77.07526241299996 -11.8439180...
3,peru_estratos_ingresos.14648,150106092000450,150106,LIMA,LIMA,CARABAYLLO,32,149,2,MULTIPOLYGON (((-77.02031777899998 -11.8968804...
4,peru_estratos_ingresos.14649,150106092000080,150106,LIMA,LIMA,CARABAYLLO,55,263,2,MULTIPOLYGON (((-77.02249469799995 -11.8977410...


In [7]:
base_merged = pd.merge(manzanas[["Mz", "T_TOTAL", "T_MUJERES", "NOMBDEP", "NOMBPROV", "NOMBDIST", "CAPITAL", 
                                "geometry"]], gdf, on='Mz', how="left")
print(base_merged.shape)
base_merged.tail()

(208165, 20)


,Mz,T_TOTAL,T_MUJERES,NOMBDEP,NOMBPROV,NOMBDIST,CAPITAL,geometry,UBIGEO,AREA,CODCCPP,ZONA,MANZANA_ID,MANZANA_A,pobdisc_mz,distancia_a_centro_disc,duracion_a_centro_disc,alta_demanda,desierto_disc_recalc,umbral_t_dinamico
208160,151007000100100017,8.0,4.0,LIMA,YAUYOS,None,None,"MULTIPOLYGON (((-75.87091 -12.34471, -75.87092...",151007,1,0001,00100,017,None,0.0,212.1272,2577.973333,0,0,20.000000
208161,151022002700000,1.0,0.0,LIMA,YAUYOS,None,None,"MULTIPOLYGON (((-76.29371 -12.52059, -76.29363...",151022,2,0027,00000,None,None,0.0,185.0066,2220.155000,0,0,37.265622
208162,151022002700000,1.0,0.0,LIMA,YAUYOS,None,None,"MULTIPOLYGON (((-76.29371 -12.52059, -76.29363...",151022,2,0027,00000,None,None,0.0,185.0066,2220.155000,0,0,37.265622
208163,151004000100100009,11.0,5.0,LIMA,YAUYOS,None,None,"MULTIPOLYGON (((-76.13725 -12.38264, -76.13743...",151004,1,0001,00100,009,None,0.0,171.4914,2057.928333,0,0,20.000000
208164,151004000100100009,11.0,5.0,LIMA,YAUYOS,None,None,"MULTIPOLYGON (((-76.13725 -12.38264, -76.13743...",151004,1,0001,00100,009,None,0.0,171.4914,2057.928333,0,0,20.000000


In [11]:
from shapely import wkt
import geopandas as gpd

# Convert the 'geom' column from WKT to shapely objects
pobreza['geom'] = pobreza['geom'].apply(wkt.loads)

# Now create the GeoDataFrame
pobreza = gpd.GeoDataFrame(pobreza, geometry='geom')

manzanas_merged = pd.merge(base_merged, pobreza[["Estrato", "geom"]], left_on='geometry', right_on="geom", how="left")
print(manzanas_merged.shape)
manzanas_merged.head()

(208165, 22)


,Mz,T_TOTAL,T_MUJERES,NOMBDEP,NOMBPROV,NOMBDIST,CAPITAL,geometry,UBIGEO,AREA,...,MANZANA_ID,MANZANA_A,pobdisc_mz,distancia_a_centro_disc,duracion_a_centro_disc,alta_demanda,desierto_disc_recalc,umbral_t_dinamico,Estrato,geom
0,150102000102100023,18.0,11.0,LIMA,LIMA,ANCON,ANCON,"MULTIPOLYGON (((-77.12763 -11.81363, -77.12723...",150102,1,...,023,None,0.0,5.4523,65.495000,0,0,20.000000,2.0,"MULTIPOLYGON (((-77.12763 -11.81363, -77.12723..."
1,150102000101101010,31.0,17.0,LIMA,LIMA,ANCON,ANCON,"MULTIPOLYGON (((-77.15185 -11.73780, -77.15151...",150102,1,...,010,None,0.0,6.2620,75.146667,0,0,20.000000,1.0,"MULTIPOLYGON (((-77.15185 -11.73780, -77.15151..."
2,150102000100600037,71.0,39.0,LIMA,LIMA,ANCON,ANCON,"MULTIPOLYGON (((-77.16978 -11.77796, -77.16981...",150102,1,...,037,None,0.0,0.9826,11.791667,0,0,20.000000,2.0,"MULTIPOLYGON (((-77.16978 -11.77796, -77.16981..."
3,150102000101202048,12.0,7.0,LIMA,LIMA,ANCON,ANCON,"MULTIPOLYGON (((-77.15222 -11.74137, -77.15140...",150102,1,...,048,None,0.0,5.6990,68.391667,0,0,20.000000,1.0,"MULTIPOLYGON (((-77.15222 -11.74137, -77.15140..."
4,150102000101202010A,2.0,2.0,LIMA,LIMA,ANCON,ANCON,"MULTIPOLYGON (((-77.16253 -11.73364, -77.16255...",150102,1,...,010,A,0.0,7.0731,84.883333,0,0,21.704531,3.0,"MULTIPOLYGON (((-77.16253 -11.73364, -77.16255..."


In [12]:
manzanas_merged = manzanas_merged.drop_duplicates()

In [13]:
manzanas_merged.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 124084 entries, 0 to 208163
Data columns (total 22 columns):
 #   Column                   Non-Null Count   Dtype   
---  ------                   --------------   -----   
 0   Mz                       124084 non-null  object  
 1   T_TOTAL                  124084 non-null  float64 
 2   T_MUJERES                124084 non-null  float64 
 3   NOMBDEP                  124084 non-null  object  
 4   NOMBPROV                 124084 non-null  object  
 5   NOMBDIST                 92376 non-null   object  
 6   CAPITAL                  92376 non-null   object  
 7   geometry                 124084 non-null  geometry
 8   UBIGEO                   124084 non-null  object  
 9   AREA                     124084 non-null  object  
 10  CODCCPP                  124084 non-null  object  
 11  ZONA                     124084 non-null  object  
 12  MANZANA_ID               121332 non-null  object  
 13  MANZANA_A                45532 non-null  

In [14]:
manzanas_merged.columns

Index(['Mz', 'T_TOTAL', 'T_MUJERES', 'NOMBDEP', 'NOMBPROV', 'NOMBDIST',
       'CAPITAL', 'geometry', 'UBIGEO', 'AREA', 'CODCCPP', 'ZONA',
       'MANZANA_ID', 'MANZANA_A', 'pobdisc_mz', 'distancia_a_centro_disc',
       'duracion_a_centro_disc', 'alta_demanda', 'desierto_disc_recalc',
       'umbral_t_dinamico', 'Estrato', 'geom'],
      dtype='object')

N° de desiertos por distrito

In [15]:
# Convert the GeoDataFrame to a Pandas DataFrame
# Note: This will drop the geometry column.
df = pd.DataFrame(manzanas_merged.drop(columns=['geometry', 'geom']))



In [16]:
df.columns

Index(['Mz', 'T_TOTAL', 'T_MUJERES', 'NOMBDEP', 'NOMBPROV', 'NOMBDIST',
       'CAPITAL', 'UBIGEO', 'AREA', 'CODCCPP', 'ZONA', 'MANZANA_ID',
       'MANZANA_A', 'pobdisc_mz', 'distancia_a_centro_disc',
       'duracion_a_centro_disc', 'alta_demanda', 'desierto_disc_recalc',
       'umbral_t_dinamico', 'Estrato'],
      dtype='object')

In [17]:
summary_table = df.groupby('NOMBDIST')['desierto_disc_recalc'].sum().reset_index()

summary_table = summary_table.sort_values(by='desierto_disc_recalc', ascending=False)

summary_table


,NOMBDIST,desierto_disc_recalc
1,ATE,737
31,SAN JUAN DE LURIGANCHO,712
17,LURIGANCHO,481
4,CARABAYLLO,401
24,PUENTE PIEDRA,341
34,SAN MARTIN DE PORRES,301
42,VILLA MARIA DEL TRIUNFO,239
0,ANCON,216
38,SANTA ROSA,213
27,RIMAC,162


In [18]:
# Create an interactive bar chart using Plotly
fig = px.bar(summary_table, x='NOMBDIST', y='desierto_disc_recalc',
             labels={'desierto_disc_recalc': 'N° de desiertos', 'NOMBDIST': 'Distrito'},
             title='N° de desiertos de cuidado por distrito (personas con discapacidad)')

# Improve layout for better readability
fig.update_layout(xaxis_tickangle=-45)

# Show the plot
fig.show()

In [19]:
summary_table = df.groupby('NOMBDEP')['desierto_disc_recalc'].sum().reset_index()

summary_table = summary_table.sort_values(by='desierto_disc_recalc', ascending=False)

summary_table.to_clipboard(index=False)

summary_table

,NOMBDEP,desierto_disc_recalc
1,LIMA,4923
0,CALLAO,666


N° personas que viven en desiertos de cuidado

In [20]:
summary_table = df.groupby('desierto_disc_recalc')['T_TOTAL'].sum().reset_index()

total_sum = summary_table['T_TOTAL'].sum()

summary_table['Porcentaje población'] = (summary_table['T_TOTAL'] / total_sum) * 100

# Assuming summary_table is your DataFrame
summary_table.to_clipboard(index=False)

summary_table



,desierto_disc_recalc,T_TOTAL,Porcentaje población
0,0,9790097.0,94.020436
1,1,622636.0,5.979564


In [22]:
summary_table = df.groupby('desierto_disc_recalc')['pobdisc_mz'].sum().reset_index()

total_sum = summary_table['pobdisc_mz'].sum()

summary_table['Porcentaje población objetivo'] = (summary_table['pobdisc_mz'] / total_sum) * 100

# Assuming summary_table is your DataFrame
summary_table.to_clipboard(index=False)

summary_table



,desierto_disc_recalc,pobdisc_mz,Porcentaje población objetivo
0,0,114380.0,85.242432
1,1,19802.0,14.757568


Según estrato de ingresos

In [23]:
summary_table = df.groupby('Estrato')['desierto_disc_recalc'].sum().reset_index()

summary_table = summary_table.sort_values(by='desierto_disc_recalc', ascending=False)

summary_table

,Estrato,desierto_disc_recalc
0,1.0,2121
1,2.0,1838
2,3.0,1097
3,4.0,269
4,5.0,44


In [25]:
# Create an interactive bar chart using Plotly
fig = px.bar(summary_table, x='Estrato', y='desierto_disc_recalc',
             labels={'desierto_disc_recalc': 'N° de desiertos', 'Estrato': 'Estrato de ingresos'},
             title='N° de desiertos de cuidado (discapacidad) por estrato de ingreso de las manzanas')

# Improve layout for better readability
fig.update_layout(xaxis_tickangle=-45)

# Show the plot
fig.show()